In [97]:
%matplotlib inline
import geopandas

madison = geopandas.read_file('madison3.shp')
madison['center'] = madison.geometry.centroid

In [98]:
madison.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYPE,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,center
0,55,025,003200,1014,550250032001014,Block 1014,G5040,U,53200,U,S,56,2324,+43.0765267,-089.4214707,"POLYGON ((-89.42162999999999 43.076766, -89.42...",POINT (-89.42143585195828 43.07721897432681)
1,55,025,001102,2007,550250011022007,Block 2007,G5040,U,53200,U,S,0,3186,+43.0761736,-089.4217068,"POLYGON ((-89.422775 43.07408299999999, -89.42...",POINT (-89.42196230307162 43.07522515737384)
2,55,025,003200,1015,550250032001015,Block 1015,G5040,R,None,None,S,0,779,+43.0780683,-089.4212601,"POLYGON ((-89.42147299999999 43.078176, -89.42...",POINT (-89.42126006660975 43.07806834494028)
3,55,025,991702,0003,550259917020003,Block 0003,G5040,R,None,None,S,0,27281334,+43.0976339,-089.4184412,"POLYGON ((-89.483766 43.09381, -89.48353 43.09...",POINT (-89.41843862544388 43.09763121731496)
4,55,025,000902,2016,550250009022016,Block 2016,G5040,U,53200,U,S,382,0,+43.0694833,-089.4194454,"POLYGON ((-89.41968199999999 43.06961, -89.419...",POINT (-89.41944540396454 43.06948329794072)


In [119]:
import itertools, networkx, pandas, shapely

# DE-9IM pattern for shared boundary between two blocks
DE9IM = 'F***1****'

G = networkx.DiGraph()

for (_, row) in madison.iterrows():
    G.add_node(row.GEOID10, geom=row.geometry,
               pos=row.center.coords[0])

_possibles, _candidates, _connections = madison.shape[0]**2, 0, 0
intersections = geopandas.sjoin(madison, madison, how="inner", op='intersects')
boundaries = list()

def shared_linear_boundary(geom1, geom2):
    intersection = geom1.intersection(geom2)
    if intersection.type in ('LineString', 'MultiLineString'):
        return intersection

    if intersection.type in ('Point', 'MultiPoint', 'Polygon', 'MultiPolygon'):
        raise Exception('{}: bad type {}'.format(repr((row1.GEOID10_left, row2.GEOID10)),
                                                 intersection.type))

    if intersection.type == 'GeometryCollection':
        types = {geom.type for geom in intersection.geoms}

        if 'Polygon' in types or 'MultiPolygon' in types:
            raise Exception('{}: type has polygon {}'.format(repr((row1.GEOID10_left, row2.GEOID10)),
                                                     repr(types)))

        if types == {'Point', 'LineString'}:
            lines = [geom for geom in intersection.geoms if geom.type == 'LineString']
            return shapely.geometry.MultiLineString(lines)

        raise Exception('{}: bad type {}'.format(repr((row1.GEOID10_left, row2.GEOID10)),
                                                 intersection.type))

for (id1, row1) in intersections.iterrows():
    _candidates += 1
    geom1, row2 = row1.geometry, madison.loc[row1.index_right]
    geom2 = row2.geometry
    if not geom1.relate_pattern(geom2, DE9IM):
        continue
    _connections += 1
    geomi = shared_linear_boundary(geom1, geom2)
    G.add_edge(row1.GEOID10_left, row2.GEOID10, line=geomi)
    boundaries.append((row1.GEOID10_left, row2.GEOID10, geomi))

print('Connections:', (_possibles, _candidates, _connections))

geo = geopandas.GeoDataFrame(boundaries, columns=('G1', 'G2', 'geom'),
                             geometry='geom', crs=madison.crs)
geo.to_file('boundaries.shp')
geo

Connections: (25600, 1174, 716)


,G1,G2,geom
0,550250032001014,550250011022006,"(LINESTRING (-89.421369 43.076503, -89.4215289..."
1,550250011022007,550250011022006,"(LINESTRING (-89.422775 43.07408299999999, -89..."
2,550250011022003,550250011022006,"(LINESTRING (-89.42159199999999 43.07569, -89...."
3,550250011022013,550250011022006,"LINESTRING (-89.418942 43.075031, -89.418926 4..."
4,550250011022012,550250011022006,"(LINESTRING (-89.418948 43.07436999999999, -89..."
5,550250011022010,550250011022006,"(LINESTRING (-89.42345899999999 43.073691, -89..."
6,550250032001006,550250011022006,"LINESTRING (-89.42167499999999 43.076501, -89...."
7,550250032001014,550250011022007,"LINESTRING (-89.42152899999999 43.0765, -89.42..."
8,550250011022006,550250011022007,"(LINESTRING (-89.42167499999999 43.076501, -89..."
9,550250011022014,550250011022003,"(LINESTRING (-89.417102 43.075501, -89.4167219..."


In [122]:
node_ids = list(G.nodes)

for node_id in node_ids:
    node_perim = G.nodes[node_id]['geom'].boundary
    for n2_id in G.neighbors(node_id):
        edge_geom = G.edges[(node_id, n2_id)]['line']
        node_perim = node_perim.difference(edge_geom)
    print(node_id, node_perim.type)
    if node_perim.type in ('LineString', 'MultiLineString'):
        G.add_edge(node_id, 'outside', line=node_perim)

550250032001014 GeometryCollection
550250011022007 GeometryCollection
550250032001015 GeometryCollection
550259917020003 MultiLineString
550250009022016 GeometryCollection
550250012003011 LineString
550250011011004 GeometryCollection
550250016032002 GeometryCollection
550250016063003 LineString
550250011011005 GeometryCollection
550250011011011 GeometryCollection
550250009022014 GeometryCollection
550250012004010 LineString
550250011022005 GeometryCollection
550250009021003 GeometryCollection
550250009021005 GeometryCollection
550250009021009 GeometryCollection
550250012004001 GeometryCollection
550250011011015 GeometryCollection
550250012003003 GeometryCollection
550250009012000 GeometryCollection
550250016064002 GeometryCollection
550250011011000 GeometryCollection
550250011012012 GeometryCollection
550250011011001 GeometryCollection
550250011022000 GeometryCollection
550250011011008 GeometryCollection
550250011012009 GeometryCollection
550250011022014 GeometryCollection
550250011011

In [123]:
import matplotlib.pyplot as plt

pos = {key: data['pos'] for (key, data) in G.nodes.items()}
labels = {key: key[-5:] for key in G.nodes}
networkx.draw_networkx_nodes(G, pos, node_color='silver', node_size=100)
networkx.draw_networkx_edges(G, pos, edge_color='silver')
networkx.draw_networkx_labels(G, pos, labels, font_color='black', font_size=9)

plt.axis('off')
plt.show()

KeyError: 'pos'

In [101]:
#madison.plot(figsize=(20, 20), facecolor='#ff9900', edgecolor='white', linewidth=1)

In [196]:
assignments = [
    '550250016062005', # <-- inside
    '550250016063000', '550250016062004', '550250016063003',
    '550250016062003', '550250016062002', '550250016051003', '550250016051004',
    '550250012001003', # <-- hole
    '550250032001014', '550250011022006' # <-- corner-touch pair
    ]

centers = geopandas.GeoDataFrame([(geoid, G.nodes[geoid]['geom'].representative_point())
                                for geoid in assignments if geoid in G.nodes],
                               columns=['G1', 'geom'],
                               geometry='geom', crs=madison.crs)

centers.to_file('district-centers.shp')

boundary = list(networkx.algorithms.boundary.edge_boundary(G, assignments))
lines = geopandas.GeoDataFrame([(g1, g2, G.edges[(g1, g2)]['line'])
                                for (g1, g2) in boundary],
                               columns=['G1', 'G2', 'geom'],
                               geometry='geom', crs=madison.crs)

lines.to_file('district-lines.shp')
lines

,G1,G2,geom
0,550250012001003,550250012004000,"(LINESTRING (-89.400745 43.06476199999999, -89..."
1,550250012001003,550250012001004,"(LINESTRING (-89.397774 43.066547, -89.397773 ..."
2,550250012001003,outside,"LINESTRING (-89.395127 43.066998, -89.395557 4..."
3,550250016063000,550250016064000,"(LINESTRING (-89.39752799999999 43.070907, -89..."
4,550250016063000,550250016063001,"(LINESTRING (-89.39757299999999 43.068959, -89..."
5,550250016063000,outside,"LINESTRING (-89.395905 43.070875, -89.39592499..."
6,550250016063003,550250016063001,"(LINESTRING (-89.39918899999999 43.0694, -89.3..."
7,550250016063003,550250016063002,"(LINESTRING (-89.400886 43.069885, -89.399591 ..."
8,550250016063003,550250011011019,"(LINESTRING (-89.40094099999999 43.06765, -89...."
9,550250016063003,550250011011016,"LINESTRING (-89.400953 43.06822, -89.400944 43..."


In [209]:
district_polygons = list(shapely.ops.polygonize(lines.geometry))

polys = geopandas.GeoDataFrame([(poly, )
                                for poly in district_polygons],
                               columns=['geom'],
                               geometry='geom', crs=madison.crs)

polys.to_file('district-polys.shp')
polys

,geom
0,"POLYGON ((-89.400745 43.06476199999999, -89.40..."
1,"POLYGON ((-89.397773 43.066501, -89.397774 43...."
2,"POLYGON ((-89.42167499999999 43.076501, -89.42..."
3,"POLYGON ((-89.42120299999999 43.077881, -89.42..."


In [200]:
multipoint = centers.geom.unary_union
polys = geopandas.GeoDataFrame([(poly, )
                                for poly in district_polygons
                                if poly.relate_pattern(multipoint, '0********')],
                               columns=['geom'],
                               geometry='geom', crs=madison.crs)

polys.to_file('district-polys.shp')
polys

,geom
0,"POLYGON ((-89.400745 43.06476199999999, -89.40..."
1,"POLYGON ((-89.42120299999999 43.077881, -89.42..."
